Beat-n game with 10 pt limit

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const MXV = 9
const MNV = 1

1

In [4]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [5]:


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [6]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get spuds with cost 10 and filter nondominated

In [7]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff = Spud("", h,f,l,p,r,s,999)
                                ff_i += 1
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 2 methods)

In [8]:
lib0 = get_library(10)
lib0

126-element Vector{Spud}:
 Spud("", 1, 1, 1, 1, 1, 5, 999)
 Spud("", 1, 1, 1, 1, 2, 4, 999)
 Spud("", 1, 1, 1, 1, 3, 3, 999)
 Spud("", 1, 1, 1, 1, 4, 2, 999)
 Spud("", 1, 1, 1, 1, 5, 1, 999)
 Spud("", 1, 1, 1, 2, 1, 4, 999)
 Spud("", 1, 1, 1, 2, 2, 3, 999)
 Spud("", 1, 1, 1, 2, 3, 2, 999)
 Spud("", 1, 1, 1, 2, 4, 1, 999)
 Spud("", 1, 1, 1, 3, 1, 3, 999)
 Spud("", 1, 1, 1, 3, 2, 2, 999)
 Spud("", 1, 1, 1, 3, 3, 1, 999)
 Spud("", 1, 1, 1, 4, 1, 2, 999)
 ⋮
 Spud("", 3, 1, 3, 1, 1, 1, 999)
 Spud("", 3, 2, 1, 1, 1, 2, 999)
 Spud("", 3, 2, 1, 1, 2, 1, 999)
 Spud("", 3, 2, 1, 2, 1, 1, 999)
 Spud("", 3, 2, 2, 1, 1, 1, 999)
 Spud("", 3, 3, 1, 1, 1, 1, 999)
 Spud("", 4, 1, 1, 1, 1, 2, 999)
 Spud("", 4, 1, 1, 1, 2, 1, 999)
 Spud("", 4, 1, 1, 2, 1, 1, 999)
 Spud("", 4, 1, 2, 1, 1, 1, 999)
 Spud("", 4, 2, 1, 1, 1, 1, 999)
 Spud("", 5, 1, 1, 1, 1, 1, 999)

In [9]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [10]:
function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

filter_nondominated2 (generic function with 1 method)

In [11]:
lib=filter_nondominated2(lib0)
for ff in lib
    print(" ")
    print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
end
nlib = length(lib)

 111124 111133 111142 111214 111223 111232 111241 111313 111322 111331 111412 111421 111511 112114 112123 112132 112141 112213 112222 112231 112312 112321 112411 113113 113122 113131 113212 113221 113311 114112 114121 114211 121114 121123 121132 121213 121222 121231 121312 121321 121411 122113 122122 122212 122221 122311 123112 123211 131113 131122 131212 131221 131311 132112 132211 141112 141211 212113 212122 212131 212212 212221 212311 213112 213121 213211 214111 222112 222211 312112 312121 312211 313111 412111

74

In [12]:
payoffs = get_payoffs(lib)

74×74 Matrix{Int64}:
  0  -1  -1  -1  -1  -1  -1   1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   0  -1  -1  -1  -1  -1  -1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   0  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   0  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   0  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   0  -1  -1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   1   0  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1   1   1   1   1   1   1   0  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1   1   1   1   1   1   1   0     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1  -1   1   1   1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1   1   1  -1   1   1   1   1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1  -1   1   1  -1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1   1  -1  -1   1   1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  ⋮ 

In [13]:
function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

spud2int (generic function with 1 method)

In [14]:
s2ind = Dict(lib[i] => i for i in 1:length(lib));
int2ind = Dict(spud2int(lib[i]) => i for i in 1:length(lib));

In [15]:
libi = [spud2int(ff) for ff in lib];

In [16]:
int2ind[313111]

73

In [17]:
as = [1,2,3]
as[end]

3

In [51]:
function eval_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    if length(bs)==0 || length(as)==0
        return 0
    end
    a = as[end]
    flag = true
    count = 0
    while flag
        if payoffs[a, bs[end-count]]==1
            count += 1
        else
            flag = false
        end
        if count == length(bs)
            flag = false
        end
    end
    return count
end

eval_beatn (generic function with 1 method)

In [52]:
as = [int2ind[ff] for ff in [214111, 112222, 313111, 113311]]
bs = [int2ind[ff] for ff in [121411, 121321, 113212, 213121]]

4-element Vector{Int64}:
 41
 40
 27
 65

In [53]:
eval_beatn(payoffs, as[1:3], bs[1:2])

2

In [54]:
eval_beatn(payoffs, bs[1:3], as[1:3])

2

In [55]:
eval_beatn(payoffs, as[1:4], bs[1:3])

2

In [56]:
eval_beatn(payoffs, bs[1:4], as[1:4])

3

In [57]:
size(payoffs)[1]

74

In [58]:
# tries all possible moves as extension of as
function try_beatn(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    ans = Array{Int64}(undef, size(payoffs)[1])
    for i in 1:size(payoffs)[1]
        as2 = copy(as)
        append!(as2, i)
        ans[i] = eval_beatn(payoffs, as2, bs)
    end
    return ans
end

try_beatn (generic function with 1 method)

In [59]:
for i in try_beatn(payoffs, as, bs)
    print(i)
end

00000000000000000000000011120110021211220011100001110000000000000120000000

In [60]:
# returns 1 if Player A lost
function lose_or_not(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    if maximum(scores) < prev_score
        return 1
    end
    return 0
end

lose_or_not (generic function with 1 method)

In [61]:
lose_or_not(payoffs, as[1:2], bs[1:2])

0

In [62]:
lose_or_not(payoffs, as[1:3], bs[1:3])

0

In [63]:
lose_or_not(payoffs, as[1:4], bs[1:4])

1

In [64]:
function legal_moves(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Array{Int64}
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return cands
end

legal_moves (generic function with 1 method)

In [65]:
# picks randomly
function choose_random(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    prev_score = eval_beatn(payoffs, bs, as)
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .>= prev_score)
    return rand(cands)
end

choose_random (generic function with 1 method)

In [66]:
# picks aggressively
function choose_aggro(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})::Int64
    scores = try_beatn(payoffs, as, bs)
    cands = findall(scores .== maximum(scores))
    return rand(cands)
end

choose_aggro (generic function with 1 method)

In [67]:
function play_random_vs_aggro(
        payoffs::Array{Int64}, 
        as::Array{Int64} = zeros(Int64, 0), 
        bs::Array{Int64} = zeros(Int64, 0))::Tuple{Int64, Array{Int64}, Array{Int64}}
    if length(as) == 0
        a = rand(1:size(payoffs)[1])
        as = append!(as, a)
    end
    if length(bs) < length(as)
        b = choose_aggro(payoffs, bs, as)
        bs = append!(bs, b)
    end
    flag = true
    a_win = 0
    while flag
        if lose_or_not(payoffs, as, bs)==1
            flag = false
            a_win = -1
        else
            # player A turn
            a = choose_random(payoffs, as, bs)
            as = append!(as, a)
        end
        if flag
            if lose_or_not(payoffs, bs, as)==1
                flag = false
                a_win = 1
            else
                # player B turn
                b = choose_aggro(payoffs, bs, as)
                bs = append!(bs, b)
            end
        end
    end
    return (a_win, as, bs)
end

function display_game(payoffs::Array{Int64}, as::Array{Int64}, bs::Array{Int64})
    for i in 1:length(as)
        print(libi[as[i]])
        print(" {")
        n = eval_beatn(payoffs, as[1:i], bs[1:(i-1)])
        print(n)
        print("}. ")
        if i <= length(bs)
            print(libi[bs[i]])
            print(" {")
            n = eval_beatn(payoffs, bs[1:i], as[1:i])
            print(n)
            print("}. ")
        end
        println()
    end
end

display_game (generic function with 1 method)

In [85]:
a_win, as, bs = play_random_vs_aggro(payoffs, [int2ind[111133]], [int2ind[222211]])
display_game(payoffs, as, bs)

111133 {0}. 222211 {1}. 
212221 {1}. 112132 {2}. 
112312 {2}. 213121 {3}. 
121222 {3}. 313111 {4}. 
113122 {4}. 


In [86]:
n_games = 100
counts = Array{Int64}(undef, (nlib, nlib)) .* 0
wins = Array{Int64}(undef, (nlib, nlib)) .* 0
for a in 1:nlib
    for b in 1:nlib
        for i in 1:n_games
            a_win, as, bs = play_random_vs_aggro(payoffs, [a], [b])
            counts[a,b] += 1
            if a_win==1
                wins[a,b] +=1
            end
        end
        
    end
end

In [90]:
maxes = [minimum(wins[i, :]) for i in 1:nlib]
for i in sortperm(-maxes)
    js = findall(wins[i, :].==minimum(wins[i, :]))
    print(libi[i])
    print(" ")
    print(libi[js])
    print(" ")
    println(minimum(wins[i, :])/n_games)
end

121321 [214111] 0.2
131221 [214111, 313111] 0.2
214111 [313111] 0.18
121312 [131212] 0.16
121411 [121321] 0.13
113311 [313111] 0.12
122311 [313111] 0.12
213112 [313111] 0.12
312112 [121321, 214111] 0.12
113212 [313111] 0.11
131212 [214111] 0.11
213121 [313111] 0.11
111133 [121321] 0.1
114112 [313111] 0.1
213211 [121321, 313111] 0.1
412111 [214111] 0.1
111331 [121321] 0.09
111421 [313111] 0.09
112312 [121321] 0.09
113131 [214111] 0.09
121123 [313111] 0.09
312121 [121321] 0.09
111232 [214111] 0.08
111412 [121321] 0.08
121231 [313111] 0.08
312211 [121321] 0.08
113122 [313111] 0.07
114121 [313111] 0.07
121222 [214111] 0.07
122212 [313111] 0.07
212113 [121321] 0.07
212122 [121321] 0.07
112132 [313111] 0.06
112213 [121321] 0.06
121132 [313111] 0.06
131122 [121312] 0.06
313111 [121321] 0.06
111223 [214111, 313111] 0.05
111313 [213112] 0.05
113221 [214111] 0.05
114211 [214111] 0.05
112321 [313111] 0.04
113113 [121321] 0.04
122221 [313111] 0.04
132211 [121321] 0.04
212131 [313111] 0.04
212212 [

## Old code

In [35]:
function get_playoffs(n_games::Int64, max_len::Int64 = 12)::Tuple{Array{Int64},Array{Int64},Array{Int64}}
    a_wins = Array{Int64}(undef, n_games)
    a_wins .= 0
    a_s = Array{Int64}(undef, (n_games, max_len))
    a_s .= 0
    b_s = Array{Int64}(undef, (n_games, max_len))
    b_s .= 0
    for i in 1:n_games
        a_win, as, bs = play_random_vs_aggro(payoffs)
        @assert(length(as) <= max_len)
        @assert(length(bs) <= max_len)
        a_wins[i] = a_win
        a_s[i, 1:length(as)] = as
        b_s[i, 1:length(bs)] = bs
    end
    return a_wins, a_s, b_s
end

get_playoffs (generic function with 2 methods)

In [36]:
@time a_wins, a_s, b_s = get_playoffs(10000, 14)

  1.279609 seconds (29.20 M allocations: 2.348 GiB, 22.32% gc time)


([-1, 1, 1, -1, 1, 1, -1, -1, 1, 1  …  -1, -1, -1, -1, 1, -1, 1, 1, 1, -1], [31 12 … 0 0; 73 45 … 0 0; … ; 35 15 … 0 0; 20 6 … 0 0], [22 16 … 0 0; 49 66 … 0 0; … ; 8 25 … 0 0; 46 11 … 0 0])

In [37]:
win_rates = [mean(a_wins[a_s[:, 1].==i].==1) for i in 1:nlib]

74-element Vector{Float64}:
 0.33653846153846156
 0.5
 0.32679738562091504
 0.33070866141732286
 0.39166666666666666
 0.49122807017543857
 0.4264705882352941
 0.3669064748201439
 0.30303030303030304
 0.4573643410852713
 0.38
 0.3819444444444444
 0.34615384615384615
 ⋮
 0.3308270676691729
 0.46621621621621623
 0.3515625
 0.36
 0.4295774647887324
 0.44029850746268656
 0.34057971014492755
 0.4453125
 0.4
 0.4649122807017544
 0.5441176470588235
 0.5422535211267606

In [38]:
for i in sortperm(-win_rates)
    print(libi[i])
    print(" ")
    println(win_rates[i])
end

131311 0.5748031496062992
121411 0.5483870967741935
131221 0.5441176470588235
313111 0.5441176470588235
412111 0.5422535211267606
121231 0.5289855072463768
131212 0.5289855072463768
121123 0.5220588235294118
114121 0.5102040816326531
123211 0.5035971223021583
122212 0.5034965034965035
111133 0.5
121213 0.5
111232 0.49122807017543857
122311 0.4852941176470588
113122 0.46923076923076923
131113 0.46875
213112 0.46621621621621623
312211 0.4649122807017544
141211 0.4605263157894737
121321 0.460431654676259
114112 0.4594594594594595
141112 0.45864661654135336
111331 0.4573643410852713
112213 0.4567901234567901
114211 0.45652173913043476
132211 0.45323741007194246
312112 0.4453125
222112 0.44029850746268656
131122 0.4297520661157025
121222 0.42962962962962964
214111 0.4295774647887324
111241 0.4264705882352941
112411 0.4195804195804196
212122 0.4161073825503356
113131 0.4153846153846154
113311 0.4067796610169492
312121 0.4
121132 0.39568345323741005
111223 0.39166666666666666
112114 0.3916666

In [52]:
i1 = sortperm(-win_rates)[3]
libi[i1]

131221

In [53]:
# win rates of p2 response to p1
counts = [sum(a_s[:, 1].==i1 .&& b_s[:, 1].== i) for i in 1:nlib]
n_p1_wins = [sum(a_s[:, 1].==i1 .&& b_s[:, 1].== i .&& a_wins .== 1) for i in 1:nlib]
win_rates2 = 1 .- (n_p1_wins .+ 1)./(counts .+ 2)

74-element Vector{Float64}:
 0.5
 0.5
 0.6
 0.5
 0.5
 0.4
 0.16666666666666663
 0.6666666666666667
 0.75
 0.25
 0.5714285714285714
 0.33333333333333337
 0.5
 ⋮
 0.2857142857142857
 0.5
 0.5
 0.4
 0.5
 0.5
 0.4
 0.5
 0.5
 0.5714285714285714
 0.5
 0.5

In [54]:
for i in sortperm(-win_rates2)[1:sum(win_rates2 .> 0.5)]
    print(libi[i])
    print(" ")
    println(win_rates2[i])
end

112132 0.8
111322 0.75
132211 0.75
212131 0.75
111313 0.6666666666666667
121411 0.6666666666666667
111142 0.6
112213 0.6
112321 0.6
114211 0.6
121132 0.6
121312 0.6
212212 0.6
111412 0.5714285714285714
112411 0.5714285714285714
312211 0.5714285714285714
